In [ ]:
from collections import Counter
import os
import matplotlib.pyplot as plt
import pickle
import xml.etree.cElementTree as ET
import re
from bs4 import BeautifulSoup
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from nltk import word_tokenize
import string
import pymorphy2
import bs4
import collections
import re
import requests
morph = pymorphy2.MorphAnalyzer()
extended_punctuation = string.punctuation + '—»«...--"''" '
from stop_words import get_stop_words
stoplist = get_stop_words('ru') + ['сей', 'свой', 'едва', 'самый', 'го', 'час', 'часы', 'ть', 'минута', 'метр'] + ['com', 'id']

In [ ]:
#функция, которая убирает из текста буквы английского и фрнцузского алфавита, стопслова и пунктуацию и лемматизирует
def preprocess_text(some_text):
    some_text = re.sub(r'\[|\]|([0-9])|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist and extended_punctuation]
    return lemmatized_text

In [ ]:
paper_corpus_lemmatized = {}
for diaries, year in zip(diaries_for_years_text.values(), range(1928, 1954)):
    years_diaries_dic[year] = [preprocess_text(diarie) for diarie in diaries]
    print(year)

In [ ]:
#создаем словарь с публицистикой, где ключа - год, значение - список с содержанием статей
papers_for_years = collections.defaultdict(list)
for file_root, dirs, files in os.walk(r'C:\Users\kryuk\Desktop\учеба\asya'):
    for file in files:
        if file.endswith('.xhtml') or file.endswith('.xml'):
            print(file)
            tree = ET.parse(os.path.join(file_root, file))
            root = tree.getroot()
            for tag in root.iter('meta'): 
                if tag.attrib['name'] == 'created' or tag.attrib['name'] == 'date': 
                    text = '\n'.join([child.text for child in list(root[1]) if child.text])
                    papers_for_years[int(re.search(r'\d{4}', tag.attrib['content'])[0])].append(text) 

In [ ]:
# for year in range(1921, 1925):
#     if year in papers_for_years:
#         print(year, '\n', papers_for_years[year])

In [ ]:
papers_for_years1 = {year: [preprocess_text(note) for note in paper] for year, paper in paper_corpus_without_punct.items()
                     if year in range(1928, 1954)}

In [ ]:
#записываем наш газетныф корпус в пикл
import pickle
with open('paper_corpus_without_punct.pickle', 'wb') as fw:
    pickle.dump(papers_for_years1, fw, protocol=2)

In [ ]:
with open('paper_corpus_lemmatized.pickle', 'rb') as f:
    paper_corpus_lemmatized = pickle.load(f)

In [ ]:
with open('paper_corpus_without_punct.pickle', 'rb') as f:
    paper_corpus_without_punct = pickle.load(f)

In [ ]:
paper_corpus_without_punct_1928 = [[word for word in paper if word not in extended_punctuation] for paper in paper_corpus_without_punct[1928]]

In [ ]:
papers_for_years_without_punkt = {year: [[word for word in paper if word not in extended_punctuation] for paper 
                                 in papers] for year, papers in paper_corpus_without_punct.items()}

In [ ]:
#уплощаем структуру словая, чтобы по значениям лежал весь мешок слов всех записей в списке
flat_dict_papers = {year: [note for paper in papers for note in paper] for year, papers in 
                    papers_for_years_without_punkt.items()}

In [ ]:
#то же делаем с корпусом дневников
flat_prozhito = {year: ' '.join([' '.join(diary) for diary in diaries]) for year, diaries in years_diaries_clean_dic.items()}

In [ ]:
flat_prozhito_without_punkt = {year: ''.join([letter for letter in string if letter not in extended_punctuation]) 
                               for year, string in years_diaries_clean_dic.items()}

In [ ]:
def get_topics(model, feature_names, no_top_words):
    '''
    берет модель топик-моделинга, слова и количество топиков, возврашает темы
    '''
    with open('topic_paper_for_1935.txt', 'w', encoding='utf-8') as f:
        for topic_idx, topic in enumerate(model.components_):
            print(f"Тема {topic_idx+1}:", file=f)
            topic_words = ", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
            print(topic_words, file=f)

In [ ]:
def main():
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, ngram_range=(2, 2), stop_words=stoplist)
    tfidf = tfidf_vectorizer.fit_transform(paper_corpus_lemmatized[1935])
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    nmf = NMF(n_components=10, random_state=42, alpha=.1, l1_ratio=.5, init='nndsvd', max_iter=100000).fit(tfidf)
    get_topics(nmf, tfidf_feature_names, 10)

In [ ]:
if __name__ == '__main__':
    main()#nndsvd

In [ ]:
import nltk
from nltk.collocations import *

In [ ]:
#ключевые слова по degree centrality, решил не использовать
import networkx as nx
def get_keywords_graph_degree_centrality(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов узлы графа с самой высокой степенью degree centrality
    """
    G = nx.Graph()
    G.add_edges_from([(text[i], text[i+1]) for i, item in enumerate(text) if i != (len(text)-1)])
    dc = nx.degree_centrality(G)
    keywords = [node for node in sorted(dc, key=dc.get, reverse=True)[:num_keywords]]
    return keywords

In [ ]:
def get_keywords_bigrams_tf_idf(texts, num_keywords):
    """
    берет список не лемматизированных текстов в виде строк
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова и биграммы, 
    отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer(stop_words=rus_stop_words, ngram_range=(1, 2))
    texts_as_tfidf_vectors = make_tf_idf.fit_transform(get_preprocessed_text(text) for text in texts)
    id2word = {i : word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0, : -(num_keywords + 1) : -1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [ ]:
for text_keywords in get_keywords_bigrams_tf_idf(texts, 10):
    print(text_keywords)

In [ ]:
# word_fd = nltk.FreqDist(flat_dict_papers[1930])
bigram_fd = nltk.FreqDist(nltk.bigrams(flat_dict_papers[1930]))
# tokens = nltk.wordpunct_tokenize(text)
finder = BigramCollocationFinder(word_fd, bigram_fd)
# sorted(finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:10]

In [ ]:
len(flat_prozhito[1930])

In [ ]:
type(flat_prozhito[1930])

In [ ]:
#функций, которая выдаёт самые частотные коллокации по убыванию
def collocater(text):
    tokens = nltk.wordpunct_tokenize(text)
    tokens_without_punkt = [token for token in tokens if token not in extended_punctuation and stoplist]
    word_prozhito = nltk.FreqDist(tokens_without_punkt)
    bigram_prozhito = nltk.FreqDist(nltk.bigrams(tokens_without_punkt))
    finder_prozhito = BigramCollocationFinder(word_prozhito, bigram_prozhito)
    finder_prozhito.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stoplist)
    finder_prozhito = sorted(finder_prozhito.ngram_fd.items(), key=lambda t: (-t[1], t[0]))
    return finder_prozhito

In [ ]:
flat_prozhito_without_punkt[1930]

In [ ]:
colloc_for_years_all = []
for key, value in flat_prozhito_without_punkt.items():
    colloc_for_years.append('\n' + str(key) + '\n')
    colloc_for_years.append(collocater(flat_prozhito_without_punkt[key]))
# with open('colloc_for_years_prozhito50.txt', 'w', encoding='utf-8') as f:
#     print(*colloc_for_years, file=f)

In [ ]:
sorted(finder_prozhito.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:30]

In [ ]:
flat_prozhito[1930][1]

In [ ]:
with open('years_diaries_clean_dic.pickle', 'rb') as f:
    years_diaries_clean_dic = pickle.load(f)

In [ ]:
sorted(finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:10]

In [ ]:
#1942 - всего 3 записи, не делает моделирование, 1935 тоже - 1
len(flat_dict_papers)

In [ ]:
for year, papers in flat_dict_papers.items():
    with open(f'Newspaper_for_{year}.txt', 'w', encoding='utf-8') as f:
        print(*papers, file=f)

In [ ]:
colloc_for_years_papers_all = []
for key, value in flat_dict_papers.items():
    colloc_for_years_papers.append('\n' + str(key) + '\n')
    colloc_for_years_papers.append(collocater(' '.join(flat_dict_papers[key]), 50))
with open('colloc_for_years_papers50.txt', 'w', encoding='utf-8') as f:
    print(*colloc_for_years_papers, file=f)

In [ ]:
type(flat_dict_papers[1928][1])

In [ ]:
authors_nkrya = []
date = None
author_nkrya = None
for file_root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.xhtml') or file.endswith('.xml'):
            tree = ET.parse(os.path.join(file_root, file))
            root = tree.getroot()
            for tag in root.iter('meta'):
                if tag.attrib['name'] in {'date', 'publ_year', 'created'}:
                    year = re.search(r'\d{4}', tag.attrib['content'])
                    if year:
                        if int(year[0]) in range(1928, 1954):
                            date = int(year[0])
                elif tag.attrib['name'] == 'author':
                    author_nkrya = tag.attrib['content']
            if date and author_nkrya:
                authors_nkrya.append(author_nkrya)
            date = None
            author_nkrya = None

In [ ]:
sorted(dict(Counter(authors_nkrya)).items(), key=lambda x: x[1], reverse=True)

In [ ]:
bigram = 'шахтинский'
bigram_for_years_prozhito = {}
for key, value in flat_prozhito_without_punkt.items():
    bigram_for_years_prozhito[key] = flat_prozhito_without_punkt[key].count(bigram)
print(bigram_for_years_prozhito)

In [ ]:
sum(list(bigram_for_years_prozhito.values()))

In [ ]:
bigram_for_years_nkrya = {}
for key, value in sorted(flat_dict_papers.items()):
    bigram_for_years_nkrya[key] = ' '.join(flat_dict_papers[key]).count(bigram)
print(bigram_for_years_nkrya)

In [ ]:
from matplotlib import style  # добавляем стили
style.use('ggplot')  # выбираем стиль ggplot

plt.plot(list(bigram_for_years_nkrya.keys()), list(bigram_for_years_nkrya.values()), 'g',label='Публицистика НКРЯ')
plt.plot(list(bigram_for_years_prozhito.keys()), list(bigram_for_years_prozhito.values()), 'r',label='"Прожито"')

plt.title(bigram)
plt.ylabel('Количество вхождений униграммы/биграммы')
plt.xlabel('Год')
plt.legend(fontsize=12)

plt.show()
plt.savefig('испания.pdf')

In [ ]:
all_topics = []
for file_root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.xhtml') or file.endswith('.xml'):
            tree = ET.parse(os.path.join(file_root, file))
            root = tree.getroot()
            for tag in root.iter('meta'):
                if tag.attrib['name'] in {'date', 'publ_year', 'created'}:
                    year = re.search(r'\d{4}', tag.attrib['content'])
                    if year:
                        if int(year[0]) in range(1928, 1954):
                            date = int(year[0])
                elif tag.attrib['name'] == 'topic':
                    topic = tag.attrib['content']
            if date and topic:
                all_topics.append(topic)
            date = None
            topic = None
sorted(dict(Counter(all_topics)).items(), key=lambda x: x[1], reverse=True)

In [ ]:
publ_places = []
for file_root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.xhtml') or file.endswith('.xml'):
            tree = ET.parse(os.path.join(file_root, file))
            root = tree.getroot()
            for tag in root.iter('meta'):
                if tag.attrib['name'] in {'date', 'publ_year', 'created'}:
                    year = re.search(r'\d{4}', tag.attrib['content'])
                    if year:
                        if int(year[0]) in range(1928, 1954):
                            date = int(year[0])
                elif tag.attrib['name'] == 'publication':
                    publ_place = tag.attrib['content']
            if date and publ_place:
                publ_places.append(publ_place)
            date = None
            publ_place = None
sorted(dict(Counter(publ_places)).items(), key=lambda x: x[1], reverse=True)